In [ ]:
!pip install adversarial-robustness-toolbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 38.3 MB/s 


Creating own network and adding the defense Certified Adversarial Robustness to it through TensorFlowV2. And evaluating the Defense using the metrics such as CAV, COS ets. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
tf.keras.backend.set_floatx('float32')
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from art import config
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.estimators.certification.randomized_smoothing import TensorFlowV2RandomizedSmoothing, RandomizedSmoothingMixin, PyTorchRandomizedSmoothing
from art.utils import load_dataset, get_file, compute_accuracy

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import argparse
import os
import random
import sys
import warnings
import torch
import torch.optim as optim
import numpy as np
import scipy.stats as jslib
import torch
import torch.nn.functional as F

from torch import nn
from sklearn.utils import shuffle

from art.estimators.certification import deep_z

In [ ]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset('mnist')

nb_classes = 10
input_shape = x_train.shape[1:]

num_samples_test = 250
x_test = x_test[0:num_samples_test].astype(np.float32)
y_test = y_test[0:num_samples_test]

x_train = x_train.astype(np.float32)

In [ ]:
def get_model(input_shape, min_, max_):
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    return model

In [ ]:
nb_epochs = 10
batch_size = 128
sample_size = 100
alpha = 0.001

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

def train_step(model, images, labels):        
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

In [ ]:
classifier = TensorFlowV2Classifier(model=get_model(input_shape, min_, max_),
                                    nb_classes=nb_classes,
                                    input_shape=input_shape,
                                    loss_object=loss_object,
                                    train_step=train_step,
                                    channels_first=False,
                                    clip_values=(min_, max_))

classifier.fit(x_train, y_train, nb_epochs=nb_epochs, batch_size=batch_size)

sigma_0 = 0.5

classifier_rs_0 = TensorFlowV2RandomizedSmoothing(model=classifier.model,
                                                  nb_classes=nb_classes,
                                                  input_shape=input_shape,
                                                  loss_object=loss_object,
                                                  train_step=train_step,
                                                  channels_first=False,
                                                  clip_values=(min_, max_),
                                                  sample_size=sample_size,
                                                  scale=sigma_0,
                                                  alpha=alpha)

In [ ]:

sigma_1 = 0.25

classifier_rs_1 = TensorFlowV2RandomizedSmoothing(model=get_model(input_shape, min_, max_),
                                                 nb_classes=nb_classes,
                                                 input_shape=input_shape,
                                                 loss_object=loss_object,
                                                 train_step=train_step,
                                                 channels_first=False,
                                                 clip_values=(min_, max_),
                                                 sample_size=sample_size,
                                                 scale=sigma_1,
                                                 alpha=alpha)

classifier_rs_1.fit(x_train, y_train, nb_epochs=nb_epochs, batch_size=batch_size)

100%|██████████| 10/10 [01:01<00:00,  6.11s/it]


In [ ]:
x_preds = classifier.predict(x_test)
x_preds_rs_1 = classifier_rs_1.predict(x_test)

Randomized smoothing:   0%|          | 0/250 [00:00<?, ?it/s]

In [ ]:
print(x_preds.shape)
print(y_test.shape)

(250, 10)
(250, 10)


In [ ]:
print(y_test[0])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [ ]:
true = []
for i in y_test:
  lab = np.where(i == 1)[0][0]
  true.append(lab)
print(true)

[7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6, 4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 6, 5, 5, 7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 9, 8, 7, 2, 3, 0, 4, 4, 2]


In [ ]:
def defense_utility_measure(pred_def, pred_raw, true_label):
    # compute the classification accuracy of raw model
    correct_prediction_raw = np.equal(np.argmax(pred_raw, axis=1), true_label)
    acc_raw = np.mean(correct_prediction_raw.astype(float))

    # compute the classification accuracy of defense-enhanced model
    correct_prediction_def = np.equal(np.argmax(pred_def, axis=1), true_label)
    acc_def = np.mean(correct_prediction_def.astype(float))

    # compute the Classification Accuracy Variance(CAV)
    cav_result = acc_def - acc_raw

    # find the index of correct predicted examples by defence-enhanced model and raw model
    idx_def = np.squeeze(np.argwhere(correct_prediction_def == True))
    idx_raw = np.squeeze(np.argwhere(correct_prediction_raw == True))
    idx = np.intersect1d(idx_def, idx_raw, assume_unique=True)

    # compute the Classification Rectify Ratio(CRR) & Classification Sacrifice Ratio(CSR)
    num_rectify = len(idx_def) - len(idx)
    crr_result = num_rectify / len(pred_def)

    num_sacrifice = len(idx_raw) - len(idx)
    csr_result = num_sacrifice / len(pred_def)

    # filter the correct prediction results
    pred_def_filter = pred_def[idx]
    pred_raw_filter = pred_raw[idx]

    # compute the Classification Confidence Variance(CCV)
    confidence_def = np.max(pred_def_filter, axis=1)
    confidence_raw = np.max(pred_raw_filter, axis=1)
    ccv_result = np.mean(np.absolute(confidence_def - confidence_raw))

    # compute the Classification Output Stability(COS)
    M = (pred_def_filter + pred_raw_filter) / 2.
    js_total = 0
    for i in range(len(M)):
        js = 0.5 * jslib.entropy(pred_def_filter[i], M[i]) + 0.5 * jslib.entropy(pred_raw_filter[i], M[i])
        js_total += js
    cos_result = js_total / len(M)

    return acc_raw, acc_def, cav_result, crr_result, csr_result, ccv_result, cos_result

In [ ]:
_,_,cav,crr,csr,ccv,cos = defense_utility_measure(x_preds_rs_1, x_preds, true)

In [ ]:
print(f'The CAV score for certified robustness enhanced model is {cav:.6f}')
print(f'The CCV score for certified robustness enhanced model is {ccv:.6f}')
print(f'The COS score for certified robustness enhanced model is {cos:.6f}')

The CAV score for certified robustness enhanced model is 0.004000
The CCV score for certified robustness enhanced model is 0.004308
The COS score for certified robustness enhanced model is 0.001695
